## SageMaker with Glue Session 
haimtran 30/05/2023
- [spark sql guide](https://spark.apache.org/docs/latest/sql-programming-guide.html)
- update execution role 
- select kernel Spark Analytics 2 and [Glue and Ray]

In [ ]:
```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "unique_statement_id",

      "Effect": "Allow",
      "Action": ["iam:GetRole", "iam:PassRole", "sts:GetCallerIdentity"],
      "Resource": "*"
    }
  ]
}
```
```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": ["glue.amazonaws.com", "sagemaker.amazonaws.com"]
      },
      "Action": "sts:AssumeRole"
    }
  ]
}
```
AwsGlueSessionUserRestrictedServiceRole

## Check Data Size 

In [ ]:
# !aws s3 ls --summarize --human-readable --recursive s3://amazon-reviews-pds/parquet/

## Configure Glue Session

In [6]:
# %additional_python_modules matplotlib, numpy, pandas, sagemaker
# %idle_timeout 60
# %glue_version 3.0
# %number_of_workers 5
# %iam_role arn:aws:iam::413175686616:role/RoleForDataScientistUserProfile

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
It looks like there is a newer version of the kernel available. The latest version is 0.38.0 and you have 0.37.4 installed.
Please run `pip install --upgrade aws-glue-sessions` to upgrade your kernel


## Create Spark Session 

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("PySparkApp").getOrCreate()

In [ ]:
print(spark.version)

## Parameters

In [ ]:
source_bucket_name = "amazon-reviews-pds"
dest_bucket_name = "sagemaker-us-east-1-413175686616"

## Read Data from S3

In [ ]:
df_parquet = spark.read.format("parquet").load(f"s3://{source_bucket_name}/parquet/")

In [ ]:
df_parquet.show(10)

In [ ]:
from pyspark.sql.types import IntegerType, StringType, StructType

schema = (
    StructType()
    .add("marketplace", StringType(), True)
    .add("customer_id", StringType(), True)
    .add("review_id", StringType(), True)
    .add("product_id", StringType(), True)
    .add("product_parent", IntegerType(), True)
    .add("product_title", StringType(), True)
    .add("product_category", StringType(), True)
    .add("star_rating", IntegerType(), True)
    .add("helpful_vote", IntegerType(), True)
    .add("total_vote", IntegerType(), True)
    .add("vine", StringType(), True)
    .add("verified_purchase", StringType(), True)
    .add("review_headline", StringType(), True)
    .add("review_body", StringType(), True)
    .add("myyear", StringType(), True)
)

inferSchema cause slow time because reading twice 
.option("inferSchema", True)

In [ ]:
df_csv = (
    spark.read.format("csv")
    .option("header", True)
    .schema(schema)
    .option("delimiter", "\t")
    .option("quote", '"')
    .load(f"s3://{source_bucket_name}/tsv/")
    # .select("marketplace", "customer_id", "review_id", "product_id", "product_title")
)

In [ ]:
df_csv.show(10)

In [ ]:
df_csv.printSchema()

## Transform 

In [ ]:
# df_clean = spark.createDataFrame(df_csv.head(1000000)).select(
#     "marketplace", "customer_id", "product_id", "star_rating"
# )

In [ ]:
# df_clean = df_csv.where("marketplace='US'").select(
#     "marketplace", "customer_id", "product_id", "star_rating"
# )

In [ ]:
df_clean = df_csv.select("marketplace", "customer_id", "product_id", "star_rating")

In [ ]:
df_clean.show(10)

## Write to S3 

In [ ]:
# !aws s3 ls

In [ ]:
df_clean.write.format("parquet").save(f"s3://{dest_bucket_name}/amazon-reviews/")

In [ ]:
!aws s3 ls --summarize --human-readable --recursive s3://sagemaker-us-east-1-413175686616/amazon-reviews/

In [ ]:
!aws s3 ls s3://sagemaker-us-east-1-413175686616/amazon-reviews/

In [ ]:
# !aws s3 rm s3://sagemaker-us-east-1-413175686616/amazon-reviews/ --recursive

## Create PySparkProcessor 

In [ ]:
!mkdir spark-code

In [ ]:
%%writefile ./spark-code/preprocess.py
import argparse

from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, StringType, StructType

# create spark session
spark = SparkSession.builder.appName("PySparkApp").getOrCreate()

# create schema
schema = (
    StructType()
    .add("marketplace", StringType(), True)
    .add("customer_id", StringType(), True)
    .add("review_id", StringType(), True)
    .add("product_id", StringType(), True)
    .add("product_parent", IntegerType(), True)
    .add("product_title", StringType(), True)
    .add("product_category", StringType(), True)
    .add("star_rating", IntegerType(), True)
    .add("helpful_vote", IntegerType(), True)
    .add("total_vote", IntegerType(), True)
    .add("vine", StringType(), True)
    .add("verified_purchase", StringType(), True)
    .add("review_headline", StringType(), True)
    .add("review_body", StringType(), True)
    .add("myyear", StringType(), True)
)


def main():
    """
    parse argument
    """
    # define parser
    parser = argparse.ArgumentParser(description="app inputs and outputs")
    parser.add_argument("--source_bucket_name", type=str, help="s3 input bucket")
    parser.add_argument("--dest_bucket_name", type=str, help="output s3 prefix")

    # parse argument
    args = parser.parse_args()
    print(f"{args.source_bucket_name} and {args.dest_bucket_name}")

    # read data from s3
    df_csv = (
        spark.read.format("csv")
        .option("header", True)
        .schema(schema)
        .option("delimiter", "\t")
        .option("quote", '"')
        .load(f"s3://{args.source_bucket_name}/tsv/")
    )

    # transform and feature engineer
    df_clean = df_csv.where("marketplace='US'").select(
        "marketplace", "customer_id", "product_id", "star_rating"
    )

    # write data to s3
    df_clean.write.format("parquet").save(
        f"s3://{args.dest_bucket_name}/amazon-reviews/"
    )


if __name__ == "__main__":
    main()